# Model Selection 모듈

- Train/Test 데이터를 분리하지 않고 머신러닝 수행

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()
dt_clf.fit(iris.data, iris.target)

DecisionTreeClassifier()

In [3]:
pred = dt_clf.predict(iris.data)
accuracy_score(iris.target, pred)

1.0

- cross_validate() method

In [4]:
from sklearn.model_selection import cross_validate

In [5]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target)

{'fit_time': array([0.00085664, 0.        , 0.00099683, 0.0009973 , 0.        ]),
 'score_time': array([0.        , 0.00099921, 0.        , 0.        , 0.        ]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])}

In [6]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target, return_train_score=True)

{'fit_time': array([0.00099611, 0.        , 0.0009973 , 0.00099897, 0.00099492]),
 'score_time': array([0.        , 0.00099707, 0.        , 0.        , 0.        ]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ]),
 'train_score': array([1., 1., 1., 1., 1.])}

- Train/Test 데이터 셋을 분리

In [7]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=2021
)

In [13]:
import pandas as pd
pd.Series(y_test).value_counts()

0    14
1    10
2     6
dtype: int64

- Stratified 분리

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=2021,
    stratify=iris.target
)
pd.Series(y_test).value_counts()

0    10
1    10
2    10
dtype: int64

- cross_val_score() method

In [15]:
from sklearn.model_selection import cross_val_score

In [16]:
# 성능 지표는 정확도(accuracy), 교차검증 세트는 5개
cross_val_score(dtc, iris.data, iris.target, cv=5)

array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ])

In [17]:
import numpy as np 
scores = cross_val_score(dtc, iris.data, iris.target, cv=5)
np.mean(scores)

0.9533333333333334

### GridSearchCV
- 교차 검증과 최적 하이퍼 파라메터 튜닝을 한꺼번에 수행

In [19]:
dtc = DecisionTreeClassifier(random_state=2021)

In [20]:
# parameter 를 dictionary 형태로 설정
params = {
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [2, 3]
}

In [21]:
from sklearn.model_selection import GridSearchCV
grid_dtc = GridSearchCV(dtc, param_grid=params, cv=3)

In [22]:
grid_dtc.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_split': [2, 3]})

In [23]:
grid_dtc.cv_results_

{'mean_fit_time': array([0.00044966, 0.00098952, 0.00098284, 0.00067465, 0.00098642,
        0.00099881, 0.00066487, 0.00066408]),
 'std_fit_time': array([4.27897169e-04, 8.02668888e-04, 1.05666034e-05, 4.77155592e-04,
        1.30010409e-05, 8.16146098e-04, 4.70134086e-04, 4.69572734e-04]),
 'mean_score_time': array([0.00066614, 0.00032616, 0.        , 0.00033243, 0.00066686,
        0.00033204, 0.        , 0.00033307]),
 'std_score_time': array([0.00047103, 0.00046126, 0.        , 0.00047013, 0.00047154,
        0.00046957, 0.        , 0.00047103]),
 'param_max_depth': masked_array(data=[2, 2, 3, 3, 4, 4, 5, 5],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'min_samples_split'

In [24]:
# 최적 파라미터
grid_dtc.best_params_

{'max_depth': 2, 'min_samples_split': 2}

In [25]:
# 최고 정확도
grid_dtc.best_score_

0.9583333333333334

In [26]:
# 최고 정확도를 가지는 최적 파라미터로 학습한 estimator
best_estimator = grid_dtc.best_estimator_
best_estimator.score(X_test, y_test)

0.9